In [1]:
import numpy as np
import pandas as pd
import subprocess as sub
import matplotlib.pyplot as plt
import time

## step 1. Generate sample set by metropolis method

In [2]:
start = time.time()
sub.run(['g++', 'eff_step1_plaquette.cpp', '-o', 'step1_e', '-I/home/hyejin/trng4-4.22', '-L/opt/trng/lib', '-ltrng4', '-std=gnu++11'])
sub.run(['./step1_e'])
print("step1 finished :", time.time()-start)

#Data of Energy&nn in fileout.txt
f = pd.read_csv('fileout_eff.txt', sep = ' ', header = 0)
size = f.s[0]
temp = f.temp[0]
step2 = f.step2[0]
J0 = np.array(f.ene)
J1 = np.array(f.nn)
J2 = np.array(f.nnn)
J3 = np.array(f.nnnn)
num = len(J0)
nth = int(f.nth[0]) #How much consideration of nearest neighbors
Jmat = np.array([J1, J2, J3]).T
target_mat = np.hstack([np.ones((num, 1)), Jmat[:, 0:nth]])
Jlist, err, _, _ = np.linalg.lstsq(target_mat, J0, rcond = None)
print("T =", temp, " Energy, J =", Jlist, " error =", err)

#Data of coefficient in filein.txt
with open("filein_eff.txt", "w") as file:
    file.write('%d\n' %size)
    file.write('%d\n' %nth)
    file.write('%d\n' %step2)
    file.write('%.10g\n' %temp)
    for i in range (nth + 1):
        data = Jlist[i]
        file.write('%.10g\n' % data)

step1 finished : 17192.301326990128
T = 4.493  Energy, J = [ 1.49246822e+01  1.09929557e+00 -2.21594646e-02 -3.22991047e-03]  error = [64569.01234103]


## step 2. optimize J, regenerate sample set by wolff method

In [ ]:
sub.run(['g++', 'eff_step2_optimize.cpp', '-o', 'step2_e', '-I/home/hyejin/trng4-4.22', '-L/opt/trng/lib','-ltrng4', '-std=gnu++11'])
for i in range (15):
    start = time.time()
    sub.run(['./step2_e'])
    print("step2 :", time.time()-start)

    #Data of Energy&nn in fileout.txt
    f = pd.read_csv('fileout_eff.txt', sep = ' ', header = 0)
    size = f.s[0]
    temp = f.temp[0]
    step2 = f.step2[0]
    J0 = np.array(f.ene)
    J1 = np.array(f.nn)
    J2 = np.array(f.nnn)
    J3 = np.array(f.nnnn)
    num = len(J0)
    nth = int(f.nth[0]) #How much consideration of nearest neighbors
    Jmat = np.array([J1, J2, J3]).T
    target_mat = np.hstack([np.ones((num, 1)), Jmat[:, 0:nth]])
    Jlist, err, _, _ = np.linalg.lstsq(target_mat, J0, rcond = None)
    print("T =", temp, " Energy, J =", Jlist, " error =", err)
    #Data of coefficient in filein.txt
    with open("filein_eff.txt", "w") as file:
        file.write('%d\n' %size)
        file.write('%d\n' %nth)
        file.write('%d\n' %step2)
        if temp>2.5: file.write('%.10g\n' %temp)
        else: file.write('%.10g\n' %(temp+0.2))
        for i in range (nth + 1):
            data = Jlist[i]
            file.write('%.10g\n' % data)

step2 : 14853.537730932236
T = 4.293  Energy, J = [ 1.70430864e+01  1.10336751e+00 -2.53168782e-02  2.34995199e-04]  error = [65110.92141222]
step2 : 13015.021592140198
T = 4.093  Energy, J = [ 1.90407261e+01  1.10918748e+00 -2.73300893e-02 -9.37409088e-04]  error = [58962.33709604]
step2 : 11162.517805099487
T = 3.8930000000000002  Energy, J = [ 2.48884996e+01  1.12379065e+00 -3.74139140e-02  1.47672201e-04]  error = [53513.23593922]
step2 : 9171.285541296005
T = 3.693  Energy, J = [ 2.82607881e+01  1.13118210e+00 -3.76686167e-02 -4.20830414e-03]  error = [55523.70869174]


In [ ]:
sub.run(['g++', 'eff_step3_at.cpp', '-o', 'step3_e', '-I/home/hyejin/trng4-4.22', '-L/opt/trng/lib','-ltrng4', '-std=gnu++11'])
sub.run(['./step3_e'])